In [1]:
import requests
res = requests.get('https://comicbus.com/html/14475.html')
# 出來會是亂碼所以加入下面這一行
res.encoding = 'cp950'
# res.text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(res.text, 'lxml')
comics = []
# 讓所有資料放入comics空的字串中
for rec in soup.select('table[id="rp_ctl05_0_dl_0"] td a'):#因為是取id所以加入#字號
    comics.append(int(rec.get('id').strip('c')))
    # strip拿掉數字前面的C,用int變成一個整數
    # print(int(rec.get('id'.strip('c'))) id是個字串先用int變成整數並切掉C

In [3]:
max(comics)
#最大集數

159

In [4]:
import pandas
comic_df = pandas.DataFrame(comics)
comic_df.columns = ['series']#集數

In [5]:
import sqlite3 as lite#建立一個SQLite數據庫

In [6]:
with lite.connect('comics.sqlite') as db:#資料庫名為 comics.sqlits 把連線設為db
    comic_df.to_sql('comics' ,con = db ,if_exists='replace')#把資料放入comics，連線設為db，表格如果存在的話做replace的動作

In [7]:
#目前在資料庫最新的集數為何
with lite.connect('comics.sqlite') as db:
    df2 = pandas.read_sql_query('select max(series) as max_series from comics', con = db)
df2

,max_series
0,159


In [8]:
# 使用def定義check_comics()
def check_comics():
# 檢查資料庫最新的漫畫集數
    with lite.connect('comics.sqlite') as db:
        df2 = pandas.read_sql_query('select max(series) as max_series from comics', con = db)
    max_series = df2['max_series'][0]# 拿出第一筆數據

# 利用網路爬蟲檢查最新的集數
    res = requests.get('https://comicbus.com/html/14475.html')# 出來會是亂碼所以加入下面這一行
    res.encoding = 'cp950'
    soup = BeautifulSoup(res.text, 'lxml')
    comics = []# 讓所有資料放入comics空的字串中
# 讓所有資料放入comics空的字串中
    for rec in soup.select('table[id="rp_ctl05_0_dl_0"] td a'):#因為是取id所以加入#字號
        comics.append(int(rec.get('id').strip('c')))
    # strip拿掉數字前面的C,用int變成一個整數
    # print(int(rec.get('id'.strip('c')))檢查是否資料有出來
# 將還未抓取的集數放到left_comics
    left_comics = []    
    for i in range(max_series, max(comics)):
        left_comice.append(i)
# 將抓取到的集數寫到資料庫中
    with lite.connect('comics.sqlite') as db:#資料庫名為 comics.sqlits 把連線設為db
        comic_df.to_sql('comics', con = db, if_exists='replace')#把資料放入comics，連線設為db，表格如果存在的話做replace的動作
    return left_comics

check_comics()

[]

# 漫畫抓取函示

In [9]:
import time, os
from selenium import webdriver
from bs4 import BeautifulSoup
import requests

def getComic(s):
    s_path = "./catoon/%s" %(s)
    driver = webdriver.Chrome('./chromedriver')
    if not os.path.exists(str(s_path)):
        os.mkdir(str(s_path))
        pageurl = 'https://comicbus.live/online/manga_14475.html?ch={}-{}'
        driver.get(pageurl.format(s,1))
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        pagenum = int(soup.select_one('#pagenum').text.split('/')[1].strip('頁')) 
        for i in range(pagenum):
            driver.get(pageurl.format(s, i+1))
            soup = BeautifulSoup(driver.page_source, 'lxml')
            imgurl = 'https:' + soup.select_one('#TheImg').get('src')
            res = requests.get(imgurl)
            with open('{}/{}.jpg'.format(s, i+ 1), 'wb') as f:
                f.write(res.content)
        time.sleep(1)
    driver.close()

In [10]:
left_comics = check_comics()
left_comics = [130]
for i in left_comics:
    getComic(i)

## 發送漫畫

In [11]:
token = 'MRkcmm4iUThiMgRtuXPdBzgTcjDepkzQNqjb1LMPdXx'
headers = {
    'Authorization': f'Bearer {token}'
}

payload = {
 'message':'最新的漫畫', 
}

files = {
    'imageFile': open('./catoon/159/1.jpg', 'rb')
}

res = requests.post('https://notify-api.line.me/api/notify', data = payload, files=files, headers = headers)

In [12]:
res.text

'{"status":200,"message":"ok"}'

In [13]:
def send_comics(f):
    token = 'MRkcmm4iUThiMgRtuXPdBzgTcjDepkzQNqjb1LMPdXx'
    headers = {
        'Authorization': f'Bearer {token}'
    }

    payload = {
     'message':'最新的漫畫', 
    }

    files = {
        'imageFile': open(f, 'rb')
    }

    res = requests.post('https://notify-api.line.me/api/notify', data = payload, files=files, headers = headers)
    return res.json()

In [ ]:
import os
left_comics = check_comics()
left_comics = [120]
for i in range(120,122):
    getComic(i)
    
    ary = []
    for f in os.listdir('./catoon/{}'.format(i)):
        ary.append(int(f.replace('.jpg', '')))
    ary.sort()
    
    for c in ary:
        print('{}/{}.jpg'.format(i,c))
        send_comics('./catoon/{}/{}.jpg'.format(i,c))

120/1.jpg
120/2.jpg
120/3.jpg
120/4.jpg
120/5.jpg
120/6.jpg
120/7.jpg
120/8.jpg
120/9.jpg
120/10.jpg
120/11.jpg
120/12.jpg
120/13.jpg
120/14.jpg
120/15.jpg
120/16.jpg
120/17.jpg
120/18.jpg
120/19.jpg
121/1.jpg
121/2.jpg
121/3.jpg
121/4.jpg
121/5.jpg
121/6.jpg
121/7.jpg
121/8.jpg
121/9.jpg
121/10.jpg
121/11.jpg
121/12.jpg
